In [1]:
import math
import os
import datetime

import numpy as np
import pandas as pd

from keras.preprocessing import image
from keras.layers import Input, Lambda, Dense, Dropout, Flatten
from keras.models import Model, Sequential

from keras.utils import to_categorical
from keras.optimizers import RMSprop, Adam

from keras.applications import xception, inception_v3

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score

from secrets import KAGGLE_USER, KAGGLE_PW

Using TensorFlow backend.


In [2]:
competition_name = 'dog-breed-identification'
data_dir = '/opt/notebooks/data/' + competition_name + '/all_train'

batch_size = 16
target_size=(299, 299)

def add_preprocess(base_model, preprocess_func, inputs_shape=(299, 299, 3)):
    inputs = Input(shape=inputs_shape)
    x = Lambda(preprocess_func)(inputs)
    outputs = base_model(x)
    model = Model(inputs, outputs)
    return model

### train

emsemble a few augmentation training data

In [3]:
base_model_x = xception.Xception(weights='imagenet', include_top=False, pooling='avg')
model_x = add_preprocess(base_model_x, xception.preprocess_input)

In [4]:
base_model_i = inception_v3.InceptionV3(weights='imagenet', include_top=False, pooling='avg')
model_i = add_preprocess(base_model_i, inception_v3.preprocess_input)

In [5]:
batches = image.ImageDataGenerator().flow_from_directory(data_dir+'/train', shuffle=False, target_size=target_size, batch_size=batch_size)
batches_test = image.ImageDataGenerator().flow_from_directory(data_dir+'/test', shuffle=False, target_size=target_size, batch_size=batch_size)

nb_batches = math.ceil(batches.n/batch_size)
nb_batches_test = math.ceil(batches_test.n/batch_size)

y_encode = batches.classes

y = to_categorical(batches.classes)

Found 10222 images belonging to 120 classes.
Found 10357 images belonging to 1 classes.


In [6]:
# bf_x_test = model_x.predict_generator(batches_test, steps=nb_batches_test, verbose=1)
# np.save(data_dir+'/results/bf_x_test', bf_x_test)
bf_x_test = np.load(data_dir+'/results/bf_x_test.npy')

In [7]:
# bf_i_test = model_i.predict_generator(batches_test, steps=nb_batches_test, verbose=1)
# np.save(data_dir+'/results/bf_i_test', bf_i_test)
bf_i_test = np.load(data_dir+'/results/bf_i_test.npy')

In [8]:
gen = image.ImageDataGenerator(rotation_range=10,
                               width_shift_range=0.1,
                               shear_range=0.15, 
                               zoom_range=0.1, 
                               channel_shift_range=10., 
                               horizontal_flip=True)

In [9]:
preds = []
nb_runs = 20
for i in range(nb_runs):
    print("i:", i)
    batches = gen.flow_from_directory(data_dir+'/train', target_size=target_size, batch_size=batch_size, shuffle=False)
    y = to_categorical(batches.classes)
    bf_x = model_x.predict_generator(batches, steps=nb_batches, verbose=1)
    
    batches = gen.flow_from_directory(data_dir+'/train', target_size=target_size, batch_size=batch_size, shuffle=False)
    bf_i = model_i.predict_generator(batches, steps=nb_batches, verbose=1)
    
    lm = Sequential([Dense(120, activation='softmax', input_shape=(2048+2048,))])
    lm.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
    lm.fit(np.hstack([bf_x, bf_i]), y, epochs=15, batch_size=nb_batches)
    
    pred = lm.predict(np.hstack([bf_x_test, bf_i_test]), batch_size=batch_size, verbose=1)
    preds.append(pred)

i: 0
Found 10222 images belonging to 120 classes.
639/639 [==============================] - 158s   
Found 10222 images belonging to 120 classes.
639/639 [==============================] - 141s   
Epoch 1/15
10222/10222 [==============================] - 0s - loss: 2.0254 - acc: 0.6610     
Epoch 2/15
10222/10222 [==============================] - 0s - loss: 0.5715 - acc: 0.8921     
Epoch 3/15
10222/10222 [==============================] - 0s - loss: 0.3729 - acc: 0.9125     
Epoch 4/15
10222/10222 [==============================] - 0s - loss: 0.2896 - acc: 0.9254     
Epoch 5/15
10222/10222 [==============================] - 0s - loss: 0.2377 - acc: 0.9347     
Epoch 6/15
10222/10222 [==============================] - 0s - loss: 0.1994 - acc: 0.9430     
Epoch 7/15
10222/10222 [==============================] - 0s - loss: 0.1728 - acc: 0.9524     
Epoch 8/15
10222/10222 [==============================] - 0s - loss: 0.1489 - acc: 0.9591     
Epoch 9/15
10222/10222 [===================

10222/10222 [==============================] - 0s - loss: 0.0718 - acc: 0.9845     
Epoch 15/15
 9632/10357 [==========================>...] - ETA: 0si: 5
Found 10222 images belonging to 120 classes.
639/639 [==============================] - 156s   
Found 10222 images belonging to 120 classes.
639/639 [==============================] - 140s   
Epoch 1/15
10222/10222 [==============================] - 0s - loss: 2.0415 - acc: 0.6609     
Epoch 2/15
10222/10222 [==============================] - 0s - loss: 0.5774 - acc: 0.8880     
Epoch 3/15
10222/10222 [==============================] - 0s - loss: 0.3743 - acc: 0.9131     
Epoch 4/15
10222/10222 [==============================] - 0s - loss: 0.2917 - acc: 0.9222     
Epoch 5/15
10222/10222 [==============================] - 0s - loss: 0.2376 - acc: 0.9340     
Epoch 6/15
10222/10222 [==============================] - 0s - loss: 0.2009 - acc: 0.9459     
Epoch 7/15
10222/10222 [==============================] - 0s - loss: 0.1741 - acc: 

10222/10222 [==============================] - 0s - loss: 0.1031 - acc: 0.9723     
Epoch 12/15
10222/10222 [==============================] - 0s - loss: 0.0916 - acc: 0.9766     
Epoch 13/15
10222/10222 [==============================] - 0s - loss: 0.0808 - acc: 0.9806     
Epoch 14/15
10222/10222 [==============================] - 0s - loss: 0.0731 - acc: 0.9829     
Epoch 15/15
 9584/10357 [==========================>...] - ETA: 0si: 10
Found 10222 images belonging to 120 classes.
639/639 [==============================] - 156s   
Found 10222 images belonging to 120 classes.
639/639 [==============================] - 139s   
Epoch 1/15
10222/10222 [==============================] - 0s - loss: 2.0409 - acc: 0.6696     
Epoch 2/15
10222/10222 [==============================] - 0s - loss: 0.5729 - acc: 0.8929     
Epoch 3/15
10222/10222 [==============================] - 0s - loss: 0.3771 - acc: 0.9072     
Epoch 4/15
10222/10222 [==============================] - 0s - loss: 0.2925 - a

10222/10222 [==============================] - 0s - loss: 0.1496 - acc: 0.9580     
Epoch 9/15
10222/10222 [==============================] - 0s - loss: 0.1317 - acc: 0.9641     
Epoch 10/15
10222/10222 [==============================] - 0s - loss: 0.1142 - acc: 0.9699     
Epoch 11/15
10222/10222 [==============================] - 0s - loss: 0.1009 - acc: 0.9762     
Epoch 12/15
10222/10222 [==============================] - 0s - loss: 0.0893 - acc: 0.9778     
Epoch 13/15
10222/10222 [==============================] - 0s - loss: 0.0793 - acc: 0.9799     
Epoch 14/15
10222/10222 [==============================] - 0s - loss: 0.0711 - acc: 0.9816     
Epoch 15/15
10176/10357 [============================>.] - ETA: 0si: 15
Found 10222 images belonging to 120 classes.
639/639 [==============================] - 156s   
Found 10222 images belonging to 120 classes.
639/639 [==============================] - 140s   
Epoch 1/15
10222/10222 [==============================] - 0s - loss: 2.0277 -

10222/10222 [==============================] - 0s - loss: 0.2352 - acc: 0.9378     
Epoch 6/15
10222/10222 [==============================] - 0s - loss: 0.2006 - acc: 0.9448     
Epoch 7/15
10222/10222 [==============================] - 0s - loss: 0.1716 - acc: 0.9522     
Epoch 8/15
10222/10222 [==============================] - 0s - loss: 0.1490 - acc: 0.9596     
Epoch 9/15
10222/10222 [==============================] - 0s - loss: 0.1315 - acc: 0.9643     
Epoch 10/15
10222/10222 [==============================] - 0s - loss: 0.1148 - acc: 0.9704     
Epoch 11/15
10222/10222 [==============================] - 0s - loss: 0.1020 - acc: 0.9741     
Epoch 12/15
10222/10222 [==============================] - 0s - loss: 0.0917 - acc: 0.9768     
Epoch 13/15
10222/10222 [==============================] - 0s - loss: 0.0802 - acc: 0.9818     
Epoch 14/15
10222/10222 [==============================] - 0s - loss: 0.0715 - acc: 0.9849     
Epoch 15/15
10096/10357 [============================>.]

In [10]:
pred_ensemble = np.stack(preds).mean(axis=0)

### predict

In [11]:
test_ids = [f.split('/')[1].split('.')[0] for f in batches_test.filenames]

In [12]:
subm=pd.DataFrame(np.hstack([np.array(test_ids).reshape(-1, 1), pred_ensemble]))
labels = pd.read_csv(data_dir+'/labels.csv')
cols = ['id']+sorted(labels.breed.unique())
subm.columns = cols

In [13]:
description = 'xception_inception_ensemble_%d_all_data_aug' % nb_runs
submission_file_name = data_dir+'/results/%s_%s.csv' % (description,
                                                        datetime.datetime.now().strftime('%Y-%m-%d-%H-%M')
                                                       )
subm.to_csv(submission_file_name, index=False)

### submit

In [14]:
!kg config -u $KAGGLE_USER -p $KAGGLE_PW -c $competition_name

In [15]:
!kg submit $submission_file_name -m $description

0.26118
